In [3]:
try:
    output_filepath
except:
    output_filepath = '../data/processed/'

In [2]:
import os
from pathlib import Path

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [7]:
df = pd.read_csv('s3://workspaces-clarity-mgmt-pro/jaime.oliver/misc/social_capital/data/raw/MIG_12082020131505678.csv')
df = df[df['Variable'] == 'Inflows of foreign population by nationality']  
df = df[df.Year == 2015]

/home/ubuntu/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
df[df.COU == 'CHN']

,CO2,Country of birth/nationality,VAR,Variable,GEN,Gender,COU,Country,YEA,Year,Value,Flag Codes,Flags


In [10]:
df[df.CO2 == 'CHN']

,CO2,Country of birth/nationality,VAR,Variable,GEN,Gender,COU,Country,YEA,Year,Value,Flag Codes,Flags
39723,CHN,China,B11,Inflows of foreign population by nationality,TOT,Total,AUS,Australia,2015,2015,27924.0,NaN,NaN
39757,CHN,China,B11,Inflows of foreign population by nationality,TOT,Total,AUT,Austria,2015,2015,2053.0,NaN,NaN
39793,CHN,China,B11,Inflows of foreign population by nationality,TOT,Total,BEL,Belgium,2015,2015,1236.0,NaN,NaN
39829,CHN,China,B11,Inflows of foreign population by nationality,TOT,Total,CAN,Canada,2015,2015,19531.0,NaN,NaN
39844,CHN,China,B11,Inflows of foreign population by nationality,TOT,Total,CZE,Czech Republic,2015,2015,469.0,NaN,NaN
39880,CHN,China,B11,Inflows of foreign population by nationality,TOT,Total,DNK,Denmark,2015,2015,1288.0,NaN,NaN
39916,CHN,China,B11,Inflows of foreign population by nationality,TOT,Total,FIN,Finland,2015,2015,732.0,NaN,NaN
39952,CHN,China,B11,Inflows of foreign population by nationality,TOT,Total,FRA,France,2015,2015,4991.0,NaN,NaN
39970,CHN,China,B11,Inflows of foreign population by nationality,TOT,Total,DEU,Germany,2015,2015,25511.0,NaN,NaN
40006,CHN,China,B11,Inflows of foreign population by nationality,TOT,Total,HUN,Hungary,2015,2015,3524.0,NaN,NaN


In [30]:
gdp = pd.read_parquet(output_filepath + '2015/industry_output.parquet')
len(gdp)

190

In [3]:
from src.data.migration_network import MigrationNetworkCreation

MNC = MigrationNetworkCreation(
        year='2010', input_filepath='../data/raw/', output_filepath=output_filepath
    )
MNC.run()

In [4]:
len(MNC.G)

180

In [13]:
MNC.G.in_edges(nbunch=['CHN'], data=True)

InEdgeDataView([])

In [8]:
import numpy as np
np.sqrt(len(MNC.G.edges(data=True)))

58.18934610390462

In [27]:
len(set(MNC.G.nodes()).intersection(set(gdp.index)))

180

In [45]:
from src.data.financial_network import IndustryNetworkCreationEORA

INC = IndustryNetworkCreationEORA(
            year='2000', input_filepath='../data/raw/', output_filepath=output_filepath
        )
INC.run()

In [9]:
len(INC.node_index)

NameError: name 'INC' is not defined

In [15]:
data_path = os.path.join('../data/raw/', 'API_SL.TLF.TOTL.IN_DS2_en_csv_v2_1929128.csv')
df_population = pd.read_csv(data_path, skiprows=4)
df_population.drop(columns=['Country Name','Indicator Name', 'Indicator Code'], inplace=True)

df_population = df_population.set_index(['Country Code']).stack().reset_index()

df_population.columns = ['country', 'year','wkn_population']

df_population['log_wkn_population'] = df_population['wkn_population'].map(lambda x: np.log(x + 1))
df_population.country.nunique()

233